In [4]:
import pandas as pd
import numpy as np
from datetime import date
import holidays

In [5]:
def generate_date_hour_dim(start_date="2023-12-01 00:00:00", end_date="2024-12-31 23:00:00"):
    """
    Generate a date-hour dimension DataFrame for a specified date range.

    Args:
        start_date (str): The start date for the date range in "YYYY-MM-DD HH:MM:SS" format.
        end_date (str): The end date for the date range in "YYYY-MM-DD HH:MM:SS" format.

    Returns:
        DataFrame: A DataFrame containing the date-hour dimension data.
    """
    
    # Generate a date range with hourly frequency
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    # Create the DateHourDim DataFrame
    date_hour_dim = pd.DataFrame(date_range, columns=['Datetime'])

    # Populate the columns
    date_hour_dim['DateHourKey'] = date_hour_dim['Datetime'].dt.strftime('%Y%m%d%H').astype(int)
    date_hour_dim['Hour'] = date_hour_dim['Datetime'].dt.hour
    date_hour_dim['TimeOfDay'] = np.where(date_hour_dim['Hour'] < 12, 'AM', 'PM')
    date_hour_dim['DayNumber'] = date_hour_dim['Datetime'].dt.dayofyear
    date_hour_dim['WeekDayNumber'] = date_hour_dim['Datetime'].dt.weekday
    date_hour_dim['WeekDayName'] = date_hour_dim['Datetime'].dt.strftime('%A')
    date_hour_dim['WeekendFlag'] = np.where(date_hour_dim['WeekDayNumber'] >= 5, 1, 0)
    date_hour_dim['MonthNumber'] = date_hour_dim['Datetime'].dt.month
    date_hour_dim['MonthName'] = date_hour_dim['Datetime'].dt.strftime('%B')
    date_hour_dim['Year'] = date_hour_dim['Datetime'].dt.year

    # Get US holidays
    us_holidays = holidays.US()

    # Function to determine if a date is a holiday
    def is_holiday(date):
        date_only = date.date()
        if date_only in us_holidays:
            return 1, us_holidays.get(date_only)
        return 0, 'None'  # Using 'None' as a placeholder for no holiday

    # Apply the holiday function to determine holiday flags and names
    date_hour_dim['HolidayFlag'], date_hour_dim['HolidayName'] = zip(*date_hour_dim['Datetime'].apply(is_holiday))

    # Ensure the WeekendFlag and HolidayFlag are binary (0 or 1)
    date_hour_dim['WeekendFlag'] = date_hour_dim['WeekendFlag'].astype(int)
    date_hour_dim['HolidayFlag'] = date_hour_dim['HolidayFlag'].astype(int)

    # Drop duplicates to ensure unique DateHourKey entries
    date_hour_dim = date_hour_dim.drop_duplicates(subset=['DateHourKey'])

    # Display the DateHourDim DataFrame (for debugging purposes)
    print(date_hour_dim)

    # Return the DateHourDim DataFrame
    return date_hour_dim


In [7]:
DateHourDim = generate_date_hour_dim(start_date="2023-01-01 00:00:00", end_date="2024-12-31 23:00:00")

                 Datetime  DateHourKey  Hour TimeOfDay  DayNumber  \
0     2023-01-01 00:00:00   2023010100     0        AM          1   
1     2023-01-01 01:00:00   2023010101     1        AM          1   
2     2023-01-01 02:00:00   2023010102     2        AM          1   
3     2023-01-01 03:00:00   2023010103     3        AM          1   
4     2023-01-01 04:00:00   2023010104     4        AM          1   
...                   ...          ...   ...       ...        ...   
17539 2024-12-31 19:00:00   2024123119    19        PM        366   
17540 2024-12-31 20:00:00   2024123120    20        PM        366   
17541 2024-12-31 21:00:00   2024123121    21        PM        366   
17542 2024-12-31 22:00:00   2024123122    22        PM        366   
17543 2024-12-31 23:00:00   2024123123    23        PM        366   

       WeekDayNumber WeekDayName  WeekendFlag  MonthNumber MonthName  Year  \
0                  6      Sunday            1            1   January  2023   
1              

/var/folders/ry/kcmcwh_95_jgg3sg1zp_pp5m0000gn/T/ipykernel_59952/2911972412.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq='H')


In [8]:
DateHourDim

,Datetime,DateHourKey,Hour,TimeOfDay,DayNumber,WeekDayNumber,WeekDayName,WeekendFlag,MonthNumber,MonthName,Year,HolidayFlag,HolidayName
0,2023-01-01 00:00:00,2023010100,0,AM,1,6,Sunday,1,1,January,2023,1,New Year's Day
1,2023-01-01 01:00:00,2023010101,1,AM,1,6,Sunday,1,1,January,2023,1,New Year's Day
2,2023-01-01 02:00:00,2023010102,2,AM,1,6,Sunday,1,1,January,2023,1,New Year's Day
3,2023-01-01 03:00:00,2023010103,3,AM,1,6,Sunday,1,1,January,2023,1,New Year's Day
4,2023-01-01 04:00:00,2023010104,4,AM,1,6,Sunday,1,1,January,2023,1,New Year's Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,2024-12-31 19:00:00,2024123119,19,PM,366,1,Tuesday,0,12,December,2024,0,None
17540,2024-12-31 20:00:00,2024123120,20,PM,366,1,Tuesday,0,12,December,2024,0,None
17541,2024-12-31 21:00:00,2024123121,21,PM,366,1,Tuesday,0,12,December,2024,0,None
17542,2024-12-31 22:00:00,2024123122,22,PM,366,1,Tuesday,0,12,December,2024,0,None


In [9]:
DateHourDim.to_csv('../data/dwh/DateHourDim.csv', index=False)